<a href="https://colab.research.google.com/github/nadirbelarouci/NN-overfitting-isgood/blob/master/NN_overfitting_is_good.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Neural Networks Overfitting is GOOD 

Overffiting may caused a lot of problems to many machine learning developers, and mostly all articles discuess how to avoid it, however in this notebook we will see what if overfitting is actually good in some senarios.

The idea is to use overfetting to memorize bytes of data into a neural network, the ordinary way that we do this is to map each index to one byte.
Our apprach here is to map multipe indices to the same byte.

what do we get from this ? 
- a lossless compression algorithm
- a read only data: any changes to the weights of the neural network will impact the data, thus assuring its integrity
- an abstraction to the type of data

The following is an example demonstrating the goal of the idea.
We will be using a neural network of 3 layers:
- the intput layer is an index mapping a byte in a 32 bit format 
- a hidden layer of size 64
- the output layer which represents the byte itself.

In [0]:
import tensorflow as tf
import numpy as np

### Preparing the data
- The data is represented as long enough string.
- The string will be represented by `x_train` and `y_train` vectors
- Each training data features are  represented as a 32 bit format of the index 
- Each training example label is the rank of the letter in the alphabet: A is 0, B is 1 etc and SPACE is 27.
- In a typical senario, the size of the possible labels is 255.

 

In [0]:
#CONSTANTS 
INPUT_LAYER_SIZE = 32
LABELS_SIZE = 27

def prepare_data():
  random_text = "BASED ON YOUR INPUT GET A RANDOM ALPHA NUHE RANDOM STRING GENERATOR CREATES A SERIES OF AND LETTERS ALRBRFNKPZNJEBPCZ FHGDQPZLHE HELLO THERE HOW ARE YOU DOING TODAY AND BLA BLA BLA"
  
  length = len(random_text)
  
  x_train = np.zeros((INPUT_LAYER_SIZE,length)) 
  y_train = np.zeros(length)                   
  
  for i in range(length):  
    x = [int(x) for x in bin(i)[2:]] # convert the index to a list of bits
    x = np.pad(x,(INPUT_LAYER_SIZE-len(x),0),'constant') # add 0 to the lists, size = INPUT_LAYER_SIZE
    
    x_train[:,i] = x
    y_train[i] = LABELS_SIZE-1 if random_text[i]==' ' else ord(random_text[i]) - 65 # map each char to its rank in the alphabet except the space
   
    
  x_train = x_train.T
  y_train = y_train.T 
  
  return x_train, y_train

In [12]:
x_train, y_train = prepare_data()
print(x_train.shape,y_train.shape)


(180, 32) (180,)


In [0]:
# Creating the neural network as described above 
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(64, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(LABELS_SIZE, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
# Train neural network until we overfitte the model
score = 0.0
while score < 1.0:
  history = model.fit(x_train, y_train, epochs=5,verbose=False)
  l = len(history.history['acc'])-1
  score = history.history['acc'][l];

In [9]:
score = model.evaluate(x_train, y_train)
print(score)


180/180 [==============================] - 0s 94us/step
[0.2428318397866355, 1.0]


In [10]:

# get the predictions     
predictions = np.argmax(model.predict(x_train),axis=1) 
# convert them to the appropriate string format 
to_char = lambda x: " " if x == LABELS_SIZE-1 else chr(x+65)
text = "".join(list(map(to_char, predictions)))

print (text)

BASED ON YOUR INPUT GET A RANDOM ALPHA NUHE RANDOM STRING GENERATOR CREATES A SERIES OF AND LETTERS ALRBRFNKPZNJEBPCZ FHGDQPZLHE HELLO THERE HOW ARE YOU DOING TODAY AND BLA BLA BLA


Eventough the proof of concept work, the neural network can memorize the data, this example needs optimisation and don't answer to the real problem of compressing data since the actual size of the neural network weights is way bigger than the actual size of the string which is (128 bytes).

Thank you for reading. 

```
Author: Nadir Belarouci
Twitter: @nadirbelarouci
Github: nadirbelarouci
```

